# Example of generating QAs for a 10K
Source: https://investors.nike.com/investors/news-events-and-reports/

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation. Furthermore, make sure you have the following packages installed:

In [1]:
# pip3 install langchain pandas pypdf

Also, make sure you have a .env file (via "`nano .env`" command in the terminal) in the root directory of this project, and then save your OpenAI API key in the `.env` file like below:
```
OPENAI_API_KEY=YOUR_API_KEY
```

### Load packages

In [2]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [3]:
from dotenv import load_dotenv
import os
import pandas as pd
from uniflow.client import Client
from uniflow.config import Config
from uniflow.model.config import OpenAIModelConfig
from langchain.document_loaders import PyPDFLoader

load_dotenv()


/opt/conda/envs/uniflow_no_installed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Prepare the input data

First, let's set current directory and input data directory, and load the raw data.

In [4]:
dir_cur = os.getcwd()
pdf_file = "nike-10k-2023.pdf"
input_file = os.path.join(f"{dir_cur}/data/raw_input/", pdf_file)

##### Load and split the pdf

In [5]:
loader = PyPDFLoader(input_file)
pages = loader.load_and_split()
page_contents = [page.page_content for page in pages]

Now we need to write a little bit prompts to generate question and answer for a given paragraph, each promopt data includes a instruction and a list of examples with "context", "question" and "answer".

In [6]:
data = [{
    "instruction": """Generate one question and its corresponding answer based on the context. Following the format of the examples below to include the same context, question, and answer in the response.""",
    "examples": [
        {
            "context": """In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.""",
            "question": """Who published A Mathematical Theory of Communication in 1948?""",
            "answer": """Claude E. Shannon."""
        },
        {
            "context": p,
            "question": """""",
            "answer": """""",
        }
    ],
} for p in page_contents[6:16] if len(p) > 200]


In [7]:
data

[{'instruction': 'Generate one question and its corresponding answer based on the context. Following the format of the examples below to include the same context, question, and answer in the response.',
  'examples': [{'context': 'In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.',
    'question': 'Who published A Mathematical Theory of Communication in 1948?',
    'answer': 'Claude E. Shannon.'},
   {'context': 'We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during

### Run the model

In this example, we will use the [OpenAIModelServer](https://github.com/CambioML/uniflow/blob/main/uniflow/model/server.py#L108) as the LLM to generate questions and answers. Let's import the config and client of this model.

In [8]:
config = Config(model_config=OpenAIModelConfig())
client = Client(config)

Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

Note sometimes the LLM doesn't return a JSON output, then uniflow will handle the failure and auto retry generating a new output.

In [9]:
output = client.run(data)

  0%|          | 0/10 [00:00<?, ?it/s]

INFO [model]: Attempt 1 failed, retrying...
 50%|█████     | 5/10 [00:47<00:38,  7.63s/it]INFO [model]: Attempt 1 failed, retrying...
INFO [model]: Attempt 2 failed, retrying...
100%|██████████| 10/10 [01:24<00:00,  8.42s/it]


### Process the output

Let's take a look of the generation output. We need to do a little postprocessing on the raw output.

In [11]:
# Extracting context, question, and answer into a DataFrame
contexts = []
questions = []
answers = []

for item in output:
    for i in item.get('output', []):
        for response in i.get('response', []):
            if any(key not in response for key in ['context', 'question', 'answer']):
                continue
            contexts.append(response['context'])
            questions.append(response['question'])
            answers.append(response['answer'])

df = pd.DataFrame({
    'Context': contexts,
    'Question': questions,
    'Answer': answers
})

In [12]:
# Set display options
pd.set_option('display.max_colwidth', None)  # or use a specific width like 50
pd.set_option('display.width', 1000)

df

,Context,Question,Answer
0,"We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth \nfiscal quarters have slightly exceeded those in the second and third fiscal quarters. However, the mix of product sales may vary \nconsiderably as a result of changes in seasonal and geographic demand for particular types of footwear , apparel and equipment, \nas well as other macroeconomic, strategic, operating and logistics-related factors.\nBecause NIKE is a consumer products company, the relative popularity and availability of various sports and fitness activities, as \nwell as changing design trends, affect the demand for our products. We must, therefore, respond to trends and shifts in consumer \npreferences by adjusting the mix of existing product offerings, developing new products, styles and categories and influencing \nsports and fitness preferences through extensive marketing. Failure to respond in a timely and adequate manner could have a \nmaterial adverse effect on our sales and profitability. This is a continuing risk. Refer to Item 1A. Risk Factors.\nOUR MARKETS\nWe report our NIKE Brand operations based on our internal geographic organization. Each NIKE Brand geographic segment \noperates predominantly in one industry: the design, development, marketing and selling of athletic footwear , apparel and \nequipment. The Company's reportable operating segments for the NIKE Brand are: North America; Europe, Middle East & Africa \n(""EMEA""); Greater China; and Asia Pacific & Latin America (""APLA""), and include results for the NIKE and Jordan brands. Sales \nthrough our NIKE Direct operations are managed within each geographic operating segment.\nConverse is also a reportable operating segment and operates predominately in one industry: the design, marketing, licensing \nand selling of casual sneakers, apparel and accessories. Converse direct to consumer operations, including digital commerce, \nare reported within the Converse operating segment results.\nUNITED STATES MARKET\nFor fiscal 2023, NIKE Brand and Converse sales in the United States accounted for approximately 43% of total revenues, \ncompared to 40% and 39% for fiscal 2022 and fiscal 2021, respectively. We sell our products to thousands of retail accounts in \nthe United States, including a mix of footwear stores, sporting goods stores, athletic specialty stores, department stores, skate, \ntennis and golf shops and other retail accounts. In the United S tates, we utilize NIKE sales offices to solicit such sales. During \nfiscal 2023, our three largest United States customers accounted for approximately 22% of sales in the United States.\nOur NIKE Direct and Converse direct to consumer operations sell our products to consumers through various digital platforms. In \naddition, our NIKE Direct and Converse direct to consumer operations sell products through the following number of retail stores \nin the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.\nNIKE, INC. 2",What are some key operating segments for the NIKE Brand?,"North America; Europe, Middle East & Africa (""EMEA""); Greater China; and Asia Pacific & Latin America (""APLA""), and include results for the NIKE and Jordan brands."
1,"INTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 

Finally, we can save the generated question answers into a `.csv` file.

In [13]:
import os

# Directory path you want to ensure exists
directory = 'data/output'

# Check if the directory exists
if not os.path.exists(directory):
    # Create the directory, including any necessary intermediate directories
    os.makedirs(directory)

In [14]:
output_df = df[['Question', 'Answer']]
output_df.to_csv("data/output/Nike_10k_QApairs.csv", index=False)